# Gerar Camada Silver
Eliminar duplicados e gerar tabela agregada

In [2]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql import functions as Func
from pyspark.sql.functions import lit

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Cria contexto Spark
spark = SparkSession.builder \
    .master('local') \
    .appName('TwitterApp') \
    .config('spark.executor.memory', '5gb') \
    .config("spark.cores.max", "6") \
    .getOrCreate()

contexto = spark.sparkContext
sqlContext = SQLContext(contexto)

df_brz = sqlContext.read.parquet('../../data/brz/parquet/tweets.parquet')
print('Bronze lido')

df_2 =  df_brz.withColumn("retweetCount",Func.col("retweetCount").cast("int")) \
          .withColumn("likeCount",Func.col("likeCount").cast("int")) 

try:
    df_svr = sqlContext.read.parquet('../../data/svr/parquet/tweets.parquet')
    print('Silver lido')
except:
    #Gravar parquet - sempre overwrite
    df_2.write.mode('overwrite').parquet('../../data/svr/parquet/tweets.parquet')
    df_2.write.partitionBy("hashtaginformada","ano","mes","dia").mode("append").parquet('../../data/svr/parquet/tweets.parquet')
    print('bronze tweets.parquet gerado')
    
v_qtd_tweets = df_brz.count()

distinct_df = df_2.distinct()

v_qtd_tweets_distincts = distinct_df.count()

print (f'Qtd registros: {v_qtd_tweets}')
print (f'Qtd registros distintos: {v_qtd_tweets_distincts}')

if v_qtd_tweets > v_qtd_tweets_distincts:
    #Gravar parquet - sempre overwrite
    distinct_df.write.mode('overwrite').parquet('../../data/svr/parquet/tweets.parquet')
    distinct_df.write.partitionBy("hashtaginformada","ano","mes","dia").mode("append").parquet('../../data/svr/parquet/tweets.parquet')
    print('bronze tweets.parquet atualiado')
else:
    print('Não ocorreu eliminação de distintos')

distinct_df.createOrReplaceTempView("tweets_view");
sql = "SELECT hashtaginformada, ano, mes, dia, \
count(*) qtd_tweets, sum(retweetCount) retweetCount, sum(likeCount) likeCount \
FROM tweets_view \
group by hashtaginformada, ano, mes, dia"

df_tweets_agg = spark.sql(sql);

#Gravar parquet - sempre overwrite
df_tweets_agg.write.mode('overwrite').parquet('../../data/svr/parquet/tweets_agg.parquet')
df_tweets_agg.write.partitionBy("hashtaginformada","ano").mode("append").parquet('../../data/svr/parquet/tweets_agg.parquet')
print('bronze tweets_agg.parquet gerado')    

#df_tweets_agg.printSchema()
df_tweets_agg.show()
spark.stop()
    

Bronze lido
Silver lido
Qtd registros: 1017
Qtd registros distintos: 1017
Não ocorreu eliminação de distintos
bronze tweets_agg.parquet gerado
+----------------+----+---+---+----------+------------+---------+
|hashtaginformada| ano|mes|dia|qtd_tweets|retweetCount|likeCount|
+----------------+----+---+---+----------+------------+---------+
|          gremio|2023|  2|  7|        97|         180|     2271|
|          gremio|2023|  2|  5|        62|         120|     1438|
|          serasa|2023|  2|  1|         3|           0|        0|
|          serasa|2023|  2|  8|         1|           0|        0|
|          gremio|2023|  2|  9|       233|         461|     6358|
|          gremio|2023|  2|  4|       358|        1432|    16881|
|          gremio|2023|  2|  6|        62|         177|     2487|
|          serasa|2023|  2|  9|         4|           0|        0|
|          serasa|2023|  2|  2|         1|           0|        0|
|          serasa|2023|  2|  3|         4|           3|        2|

In [2]:
df_distinct.printSchema()

NameError: name 'df_distinct' is not defined

In [3]:
svr_tweets_parquet = distinct_df

In [4]:
svr_tweets_parquet.printSchema()

root
 |-- ano: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- dia: integer (nullable = true)
 |-- hashtaginformada: string (nullable = true)
 |-- id: long (nullable = true)
 |-- retweetCount: integer (nullable = true)
 |-- likeCount: integer (nullable = true)
 |-- lang: string (nullable = true)
 |-- sourceLabel: string (nullable = true)
 |-- texto: string (nullable = true)
 |-- date: string (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- location: string (nullable = true)
 |-- displayname: string (nullable = true)
 |-- username: string (nullable = true)
 |-- followersCount: long (nullable = true)
 |-- friendsCount: long (nullable = true)



In [6]:
svr_tweets_agg = df_tweets_agg
svr_tweets_agg.printSchema()

root
 |-- hashtaginformada: string (nullable = true)
 |-- ano: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- dia: integer (nullable = true)
 |-- qtd_tweets: long (nullable = false)
 |-- retweetCount: long (nullable = true)
 |-- likeCount: long (nullable = true)

